First: import the requested packages

In [334]:
import autograd.numpy as np
import autograd as ag
import time
from matplotlib import pyplot as plt
import seaborn as sns

Define the function to be minimize

f: $R^{N}$ --> $R$

In [353]:
n=  2                                                 

def f(x):
    return x[0]**2+ x[1]**2

Define the inequality constraints functions ( the number of constraints is M, every constraints need to be in the form **g(x) <= a**)

g: $R^{N}$ --> $R^{M}$

In [354]:
ng = 5  # Number of inequality constraints                                                                              
def gv(x): 
    g1= 1- x[0]-x[1]
    g2= 1- x[0]**2 -x[1]**2
    g3= 9 - 9*x[0]**2 -x[1]**2
    g4= x[1]-x[0]**2
    g5= x[0]-x[1]**2
    
    return ([g1,g2,g3,g4,g5])

Define the equality constraints functions ( the number of constraints is L, every constraints need to be in the form h(x) = 0)

h: $R^{N}$ --> $R^{L}$

In [355]:
nh = 0   # Number of equality constrains

def hv(x):                                                                                             
    h1= 0
    h2= 0
    
    return np.array([h1,h2])

Now we define a the function that solves an unconstrained optimization problem. This function will be used in the Penalty Method. ( this function is an implementation of the Truncated-Newton-Method with Conjugate-Gradient search direction. ) 

In [356]:

def ottnonvinc(f,x):
    g = ag.grad(f)
    h = ag.hessian(f)
    g_toll = 10e-04

    delta = 0.5
    gamma = 10e-3
    eps1 = 10e-4
    eps2 = 10e-4

    k=0
    #CONJUGATE-GRADIENT
    while np.linalg.norm(g(x,eps))> g_toll:
        
        
        d=0
        b=0
        s= -g(x,eps)
        if np.dot(np.dot(s,h(x,eps)),s)< eps1*(np.linalg.norm(s))**2:
            d= -g(x,eps)
        else:
            a= -(np.dot((np.dot(h(x,eps),d)+g(x,eps)),s))/(np.dot(np.dot(s,h(x,eps)),s))
            d= d+ a*s
            while np.linalg.norm(np.dot(h(x,eps),d)+g(x,eps)) > (1/(k+1))*eps2*np.linalg.norm(g(x,eps)):
                b= np.dot(np.dot((np.dot(h(x,eps),d)+g(x,eps)),h(x,eps)),s)/(np.dot(np.dot(s,h(x,eps)),s))
                s = -(np.dot(h(x,eps),d)+g(x,eps)) + b*s
                if np.dot(np.dot(s,h(x,eps)),s) < eps1*(np.linalg.norm(s))**2:
                    break
                else:
                    a = -(np.dot((np.dot(h(x,eps),d)+g(x,eps)),s))/(np.dot(np.dot(s,h(x,eps)),s))
                    d = d + a*s
        #ARMIJO-CONDITION
        armijo = 1
        while f(x + armijo * d,eps) > f(x,eps) + gamma * armijo * (np.dot(g(x,eps), d)):
            armijo = delta * armijo
        x = x+ armijo*d
        k+=1
        #Consider to set some early stopping criteria
            
        
    return (x)

Now we define the following functions

In [357]:
def isfeasg(x):                            # is x feasible for g inequality constraints?
    r = gv(x)
    if max(r) > 10**-3:
        return False
    else:
        return True


def isfeash(x):                           # is x feasible for h equality constraints?
    r = hv(x)
    if np.linalg.norm(max(r)) > 10**-3:
        return False
    else:
        return True


def complementarity(x, lambd, ng):        # Complementarity of Lagrange multiplier and g
    r = []
    for i in range(ng):
        r.append(lambd[i] * gv(x)[i])
    for i in range(ng):
        if np.linalg.norm(r[i]) > 10**-3:
            return False
    return True


def lambdpos(lambd):                 # positivity of lagrange multiplier
    for i in range(ng):
        if lambd[i] < -10**-3:
            return False
    return True

def Lagrangiana(x,lambd,mu):          # Lagrangian Function
    l=f(x)
    for i in range(ng):
        l+= lambd[i]*gv(x)[i]
    for i in range(nh):
        l+= mu[i]*hv(x)[i]
    return l

def gradLagrangiana(x,lambd,mu):          # Gradient of Lagrangian Function
    l= ag.grad(Lagrangiana)(x,lambd,mu)
    for i in range(n):
        if np.linalg.norm(l[i]) > 10**-3:
            return False
    return True

Now we define the KKT conditions function

In [358]:
def iskkt(x, lambd, mu, ng):
    if isfeasg(x) == True and isfeash(x) == True and complementarity(x, lambd, ng) == True and lambdpos(lambd) == True and gradLagrangiana(x,lambd,mu) == True :
        return True
    return False

Now we define the following functions

In [359]:
def penalità(x,eps):                         # The penalty function to be optimized by the unconstrained optimization algorithm in the Penalty Method
    p=f(x)
    for i in range(ng):
        p += (1/eps)*(max(0,gv(x)[i]))**2
    for i in range(nh):
        p += (1/eps)*(hv(x)[i])**2
    return p

def PHI(x):                                 # this functions will be used to decrease eps in the Penalty Method
    r=0
    for i in range(ng):
        r += (max(0,gv(x)[i]))**2
    for i in range(nh):
        r+= (hv(x)[i])**2
    return r

Defining the initial point x and other needed constants

In [360]:
x = np.array([22.3,0.5])                                                                                                             # DA CAMBIARE SEMPRE
eps =  0.01  
teta1= 0.5
teta2= 0.9
teta3= 0.05

lambd= []
for i in range(ng):
    lambd.append(max(0,gv(x)[i]))
lambd= (2/eps)*np.array(lambd)

mu= []
for i in range(nh):
    mu.append(hv(x)[i])
mu= (2/eps)*np.array(mu)

Finally we define the Main Penalty Method Algorithm

In [361]:
while iskkt(x,lambd,mu,ng) != True:
    xprima= x
    x= ottnonvinc(penalità,x)
    is_decreased=0
    if PHI(x)<= PHI(xprima)*teta1:
        pass
    else:
        eps = eps*teta2
        is_decreased=1
        if eps<0.000001:
            break
    lambd = []
    for i in range(ng):
        lambd.append(max(0, gv(x)[i]))
    lambd = (2 / eps) * np.array(lambd)
    mu = []
    for i in range(nh):
        mu.append(hv(x)[i])
    mu = (2 / eps) * np.array(mu)
    if np.linalg.norm(xprima-x)<10**-10 and is_decreased!=1:

        break

print("final point:", x)
print("Function value:", f(x))
print("")
print(isfeasg(x))
print(gv(x))
print(isfeash(x))
print(complementarity(x,lambd,ng))
print(lambdpos(lambd))
print(lambd)


final point: [0.99999896 0.99999896]
Function value: 1.9999958201665569

True
[-0.9999979100821865, -0.9999958201665569, -0.9999791008327835, 1.0449578148330474e-06, 1.0449578148330474e-06]
True
True
True
[0.         0.         0.         2.22222471 2.22222471]
